<a href="https://colab.research.google.com/github/Debbiekj/Neural-Language-Models/blob/main/Neural_Language%C2%A0Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Word2Vec**

In [163]:
pip install gensim scikit-learn

In [164]:
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [165]:
sentences = [["I", "Like", "You"], ["You", "Are", "Amazing"]]

In [166]:
# min_count indicates word's minimum frequency.
model = Word2Vec(sentences, min_count=1)

In [167]:
vectors = model.wv["You"]
print(vectors)


[-5.3622725e-04  2.3643136e-04  5.1033497e-03  9.0092728e-03
 -9.3029495e-03 -7.1168090e-03  6.4588725e-03  8.9729885e-03
 -5.0154282e-03 -3.7633716e-03  7.3805046e-03 -1.5334714e-03
 -4.5366134e-03  6.5540518e-03 -4.8601604e-03 -1.8160177e-03
  2.8765798e-03  9.9187379e-04 -8.2852151e-03 -9.4488179e-03
  7.3117660e-03  5.0702621e-03  6.7576934e-03  7.6286553e-04
  6.3508903e-03 -3.4053659e-03 -9.4640139e-04  5.7685734e-03
 -7.5216377e-03 -3.9361035e-03 -7.5115822e-03 -9.3004224e-04
  9.5381187e-03 -7.3191668e-03 -2.3337686e-03 -1.9377411e-03
  8.0774371e-03 -5.9308959e-03  4.5162440e-05 -4.7537340e-03
 -9.6035507e-03  5.0072931e-03 -8.7595852e-03 -4.3918253e-03
 -3.5099984e-05 -2.9618145e-04 -7.6612402e-03  9.6147433e-03
  4.9820580e-03  9.2331432e-03 -8.1579173e-03  4.4957981e-03
 -4.1370760e-03  8.2453608e-04  8.4986202e-03 -4.4621765e-03
  4.5175003e-03 -6.7869602e-03 -3.5484887e-03  9.3985079e-03
 -1.5776526e-03  3.2137157e-04 -4.1406299e-03 -7.6826881e-03
 -1.5080082e-03  2.46979

**Consine** **similarity** **between** **the** **sentences**

In [168]:
# Get the word vectors for each word in the sentences
sentence_vectors = []
for sentence in sentences:
    vectors = [model.wv[word] for word in sentence if word in model.wv]
    if vectors:
        sentence_vectors.append(np.mean(vectors, axis=0))
    else:
        # In case none of the words in the sentence are in the model's vocabulary
        sentence_vectors.append(np.zeros(model.vector_size))

# Calculate the cosine similarity between the sentences
similarity = cosine_similarity([sentence_vectors[0]], [sentence_vectors[1]])

print("Cosine similarity:", similarity[0][0])



Cosine similarity: 0.3679116


### LSTM-based language **model**

In [169]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import numpy as np
from keras.utils import to_categorical


In [170]:
text = [
    "I love donuts",
    "Mangoes are my favourite fruit",
    "I exercise everyday"]

In [171]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)
vocab_size = len(tokenizer.word_index) + 1

In [172]:
input_sequences = []
output_words = []
for sequence in sequences:
    for i in range(1, len(sequence)):
        input_sequences.append(sequence[:i])
        output_words.append(sequence[i])


In [173]:
max_sequence_length = max([len(seq) for seq in input_sequences])
padded_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length)


In [174]:
X = np.array(padded_sequences)
y = to_categorical(output_words, num_classes=vocab_size)


X = np.reshape(X, (X.shape[0], X.shape[1], 1))


In [175]:
model = Sequential()
model.add(LSTM(128, input_shape=(max_sequence_length, 1), activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 2.3945 - accuracy: 0.0000e+00
Epoch 2/100
1/1 [==============================] - 0s 16ms/step - loss: 2.3778 - accuracy: 0.0000e+00
Epoch 3/100
1/1 [==============================] - 0s 14ms/step - loss: 2.3613 - accuracy: 0.0000e+00
Epoch 4/100
1/1 [==============================] - 0s 15ms/step - loss: 2.3451 - accuracy: 0.1250
Epoch 5/100
1/1 [==============================] - 0s 16ms/step - loss: 2.3290 - accuracy: 0.1250
Epoch 6/100
1/1 [==============================] - 0s 17ms/step - loss: 2.3131 - accuracy: 0.1250
Epoch 7/100
1/1 [==============================] - 0s 18ms/step - loss: 2.2972 - accuracy: 0.1250
Epoch 8/100
1/1 [==============================] - 0s 22ms/step - loss: 2.2814 - accuracy: 0.1250
Epoch 9/100
1/1 [==============================] - 0s 19ms/step - loss: 2.2656 - accuracy: 0.1250
Epoch 10/100
1/1 [==============================] - 0s 14ms/step - loss: 2.2497 - accuracy: 0.1250
Epoch 11/

In [176]:
seed_text = "I love"
seed_sequence = tokenizer.texts_to_sequences([seed_text])[0]
padded_seed_sequence = pad_sequences([seed_sequence], maxlen=max_sequence_length)
predicted_probabilities = model.predict(padded_seed_sequence, verbose=0)
predicted_word_index = np.argmax(predicted_probabilities)
predicted_word = tokenizer.index_word[predicted_word_index]
print("Generated Text:", seed_text + " " + predicted_word)



Generated Text: I love donuts


### GRU-based language **model**

In [177]:
from keras.models import Sequential
from keras.layers import Embedding, GRU, Dense
from keras.utils import pad_sequences
from keras.utils import to_categorical

In [178]:
my_sentences = [
    "I love birds",
    "They are amazing",

 ]

# Convert sentences to sequences of integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(my_sentences)
sequences = tokenizer.texts_to_sequences(my_sentences)

# Pad sequences to ensure equal length
max_sequence_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Prepare input and output data
X = padded_sequences[:, :-1]
y = to_categorical(padded_sequences[:, -1], num_classes=len(tokenizer.word_index) + 1)

X = np.reshape(X, (X.shape[0], X.shape[1], 1))


In [179]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_sequence_length - 1))
model.add(GRU(100))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(X, y, epochs=50)

Epoch 1/50
1/1 [==============================] - 3s 3s/step - loss: 1.9609
Epoch 2/50
1/1 [==============================] - 0s 15ms/step - loss: 1.9340
Epoch 3/50
1/1 [==============================] - 0s 13ms/step - loss: 1.9073
Epoch 4/50
1/1 [==============================] - 0s 13ms/step - loss: 1.8804
Epoch 5/50
1/1 [==============================] - 0s 13ms/step - loss: 1.8533
Epoch 6/50
1/1 [==============================] - 0s 13ms/step - loss: 1.8257
Epoch 7/50
1/1 [==============================] - 0s 13ms/step - loss: 1.7975
Epoch 8/50
1/1 [==============================] - 0s 13ms/step - loss: 1.7685
Epoch 9/50
1/1 [==============================] - 0s 13ms/step - loss: 1.7384
Epoch 10/50
1/1 [==============================] - 0s 14ms/step - loss: 1.7071
Epoch 11/50
1/1 [==============================] - 0s 12ms/step - loss: 1.6744
Epoch 12/50
1/1 [==============================] - 0s 13ms/step - loss: 1.6402
Epoch 13/50
1/1 [==============================] - 0s 13ms/step

In [180]:
seed_text = "I love"
seed_sequence = tokenizer.texts_to_sequences([seed_text])[0][-max_sequence_length+1:]
padded_seed_sequence = pad_sequences([seed_sequence], maxlen=max_sequence_length-1)
predicted_probabilities = model.predict(padded_seed_sequence, verbose=0)
predicted_word_index = np.argmax(predicted_probabilities)
predicted_word = tokenizer.index_word[predicted_word_index]
generated_text = seed_text + " " + predicted_word
print("Generated Text:", generated_text)




Generated Text: I love birds


### **Sequence-to-Sequence** **Model**

In [181]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, MultiHeadAttention, Dense
from tensorflow.keras.models import Model

In [182]:
# Define the model parameters
vocab_size = 10000
max_sequence_length = 50
embedding_dim = 256
num_heads = 8
hidden_dim = 512

# Define the encoder input
encoder_input = Input(shape=(max_sequence_length,))

# Embedding layer
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_input)

# Multi-head self-attention layer
encoder_attention = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(encoder_embedding, encoder_embedding)
encoder_output = tf.keras.layers.GlobalAveragePooling1D()(encoder_attention)

# Feed-forward network
encoder_output = Dense(hidden_dim, activation='relu')(encoder_output)

# Define the decoder input
decoder_input = Input(shape=(max_sequence_length,))

# Embedding layer
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_input)

# Multi-head self-attention layer
decoder_attention = MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)(decoder_embedding, decoder_embedding)
decoder_output = tf.keras.layers.GlobalAveragePooling1D()(decoder_attention)

# Feed-forward network
decoder_output = Dense(hidden_dim, activation='relu')(decoder_output)

# Define the output layer
output = Dense(vocab_size, activation='softmax')(decoder_output)

In [183]:
# Create the model
model = Model(inputs=[encoder_input, decoder_input], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [184]:
# Print the model summary
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 50)]         0           []                               
                                                                                                  
 embedding_13 (Embedding)       (None, 50, 256)      2560000     ['input_7[0][0]']                
                                                                                                  
 multi_head_attention_4 (MultiH  (None, 50, 256)     2103552     ['embedding_13[0][0]',           
 eadAttention)                                                    'embedding_13[0][0]']           
                                                                                                  
 global_average_pooling1d_4 (Gl  (None, 256)         0           ['multi_head_attention_4[0]